In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
%matplotlib inline


# Load the dataset
in_file = '/kaggle/input/pakistan-car-reviews/Pakistan Car Reviews.csv'

data = pd.read_csv(in_file,encoding='mac_roman')

data.head()

,Car Model,Review,Style,Comfort,Fuel Economy,Performance,Value for Money
0,2021 United Alpha 1.0 Manual,Exterior: Beautiful Look from front and back b...,5-rating,5-rating,5-rating,5-rating,5-rating
1,2009 Honda City 1.3 i-VTEC Prosmatec,Do you find this review helpful? YesNo Report ...,5-rating,5-rating,5-rating,5-rating,5-rating
2,2009 Honda City 1.3 i-VTEC Prosmatec,Good car but style is old and fuel economy is ...,5-rating,5-rating,5-rating,3-rating,5-rating
3,2021 KIA Picanto 1.0 MT,Do you find this review helpful? YesNo Report ...,5-rating,5-rating,2-rating,5-rating,4-rating
4,2009 Toyota Corolla Altis SR Cruisetronic 1.8,Good car but style is old and fuel economy is ...,5-rating,4-rating,3-rating,3-rating,5-rating


In [4]:
!pip install nltk

In [5]:
isnull = data.isnull().sum()
isnull.head()

Car Model       0
Review          0
Style           0
Comfort         0
Fuel Economy    0
dtype: int64

In [6]:
replacement = {
    "-rating": "",
}

data['Style'] =    data['Style'].replace(replacement, regex=True)
data['Comfort'] =data['Style'].replace(replacement, regex=True)
data['Fuel Economy']= data['Style'].replace(replacement, regex=True)
data['Performance'] = data['Style'].replace(replacement, regex=True)
data['Value for Money'] = data['Style'].replace(replacement, regex=True)
data

,Car Model,Review,Style,Comfort,Fuel Economy,Performance,Value for Money
0,2021 United Alpha 1.0 Manual,Exterior: Beautiful Look from front and back b...,5,5,5,5,5
1,2009 Honda City 1.3 i-VTEC Prosmatec,Do you find this review helpful? YesNo Report ...,5,5,5,5,5
2,2009 Honda City 1.3 i-VTEC Prosmatec,Good car but style is old and fuel economy is ...,5,5,5,5,5
3,2021 KIA Picanto 1.0 MT,Do you find this review helpful? YesNo Report ...,5,5,5,5,5
4,2009 Toyota Corolla Altis SR Cruisetronic 1.8,Good car but style is old and fuel economy is ...,5,5,5,5,5
...,...,...,...,...,...,...,...
2803,Honda Civic,"Chevrolet Joy is a good balanced, roomy and a...",5,5,5,5,5
2804,Suzuki Baleno,Do you find this review helpful? YesNo Report ...,5,5,5,5,5
2805,Toyota Corolla,"an excellent car, in 1000cc catagory...",4,4,4,4,4
2806,Suzuki Mehran,Do you find this review helpful? YesNo Report ...,2,2,2,2,2


In [7]:
data.dtypes

Car Model          object
Review             object
Style              object
Comfort            object
Fuel Economy       object
Performance        object
Value for Money    object
dtype: object

In [8]:
data['Comfort']  = data['Comfort'].astype('int')
data['Fuel Economy']  = data['Fuel Economy'].astype('int')
data['Performance']  = data['Performance'].astype('int')
data['Value for Money']  = data['Value for Money'].astype('int')
data['Style']  = data['Style'].astype('int')
data.dtypes

Car Model          object
Review             object
Style               int64
Comfort             int64
Fuel Economy        int64
Performance         int64
Value for Money     int64
dtype: object

In [9]:
data['totalratings'] = data['Value for Money'] + data['Performance'] + data['Comfort'] + data['Style'] + data['Fuel Economy']

In [10]:
data

,Car Model,Review,Style,Comfort,Fuel Economy,Performance,Value for Money,totalratings
0,2021 United Alpha 1.0 Manual,Exterior: Beautiful Look from front and back b...,5,5,5,5,5,25
1,2009 Honda City 1.3 i-VTEC Prosmatec,Do you find this review helpful? YesNo Report ...,5,5,5,5,5,25
2,2009 Honda City 1.3 i-VTEC Prosmatec,Good car but style is old and fuel economy is ...,5,5,5,5,5,25
3,2021 KIA Picanto 1.0 MT,Do you find this review helpful? YesNo Report ...,5,5,5,5,5,25
4,2009 Toyota Corolla Altis SR Cruisetronic 1.8,Good car but style is old and fuel economy is ...,5,5,5,5,5,25
...,...,...,...,...,...,...,...,...
2803,Honda Civic,"Chevrolet Joy is a good balanced, roomy and a...",5,5,5,5,5,25
2804,Suzuki Baleno,Do you find this review helpful? YesNo Report ...,5,5,5,5,5,25
2805,Toyota Corolla,"an excellent car, in 1000cc catagory...",4,4,4,4,4,20
2806,Suzuki Mehran,Do you find this review helpful? YesNo Report ...,2,2,2,2,2,10


In [11]:
data=data[~data['Review'].str.contains('Do you find this review helpful')]
data

,Car Model,Review,Style,Comfort,Fuel Economy,Performance,Value for Money,totalratings
0,2021 United Alpha 1.0 Manual,Exterior: Beautiful Look from front and back b...,5,5,5,5,5,25
2,2009 Honda City 1.3 i-VTEC Prosmatec,Good car but style is old and fuel economy is ...,5,5,5,5,5,25
4,2009 Toyota Corolla Altis SR Cruisetronic 1.8,Good car but style is old and fuel economy is ...,5,5,5,5,5,25
6,2020 Hyundai Santa Fe GLS,After doing a lot of research in the market fo...,4,4,4,4,4,20
8,2021 Proton X70 Premium FWD,Stylish exterior. Body is more stronger than a...,3,3,3,3,3,15
...,...,...,...,...,...,...,...,...
2799,Chevrolet Joy 1.0,"Plus: 1. Spacious, for a 800cc car. I've drov...",1,1,1,1,1,5
2801,Daihatsu Cuore,I feel it's an amazing ride inspired by its ol...,5,5,5,5,5,25
2803,Honda Civic,"Chevrolet Joy is a good balanced, roomy and a...",5,5,5,5,5,25
2805,Toyota Corolla,"an excellent car, in 1000cc catagory...",4,4,4,4,4,20


In [12]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [13]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

/opt/conda/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [14]:
data=data.reset_index(drop=True)
data

,Car Model,Review,Style,Comfort,Fuel Economy,Performance,Value for Money,totalratings
0,2021 United Alpha 1.0 Manual,Exterior: Beautiful Look from front and back b...,5,5,5,5,5,25
1,2009 Honda City 1.3 i-VTEC Prosmatec,Good car but style is old and fuel economy is ...,5,5,5,5,5,25
2,2009 Toyota Corolla Altis SR Cruisetronic 1.8,Good car but style is old and fuel economy is ...,5,5,5,5,5,25
3,2020 Hyundai Santa Fe GLS,After doing a lot of research in the market fo...,4,4,4,4,4,20
4,2021 Proton X70 Premium FWD,Stylish exterior. Body is more stronger than a...,3,3,3,3,3,15
...,...,...,...,...,...,...,...,...
1400,Chevrolet Joy 1.0,"Plus: 1. Spacious, for a 800cc car. I've drov...",1,1,1,1,1,5
1401,Daihatsu Cuore,I feel it's an amazing ride inspired by its ol...,5,5,5,5,5,25
1402,Honda Civic,"Chevrolet Joy is a good balanced, roomy and a...",5,5,5,5,5,25
1403,Toyota Corolla,"an excellent car, in 1000cc catagory...",4,4,4,4,4,20


In [15]:
data['positivity_in_review'] = 0.0
for index, row in data.iterrows():
    score=sia.polarity_scores(row['Review'])
    data.positivity_in_review.iloc[index] = score['pos']
data

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Car Model,Review,Style,Comfort,Fuel Economy,Performance,Value for Money,totalratings,positivity_in_review
0,2021 United Alpha 1.0 Manual,Exterior: Beautiful Look from front and back b...,5,5,5,5,5,25,0.296
1,2009 Honda City 1.3 i-VTEC Prosmatec,Good car but style is old and fuel economy is ...,5,5,5,5,5,25,0.128
2,2009 Toyota Corolla Altis SR Cruisetronic 1.8,Good car but style is old and fuel economy is ...,5,5,5,5,5,25,0.128
3,2020 Hyundai Santa Fe GLS,After doing a lot of research in the market fo...,4,4,4,4,4,20,0.150
4,2021 Proton X70 Premium FWD,Stylish exterior. Body is more stronger than a...,3,3,3,3,3,15,0.387
...,...,...,...,...,...,...,...,...,...
1400,Chevrolet Joy 1.0,"Plus: 1. Spacious, for a 800cc car. I've drov...",1,1,1,1,1,5,0.071
1401,Daihatsu Cuore,I feel it's an amazing ride inspired by its ol...,5,5,5,5,5,25,0.171
1402,Honda Civic,"Chevrolet Joy is a good balanced, roomy and a...",5,5,5,5,5,25,0.211
1403,Toyota Corolla,"an excellent car, in 1000cc catagory...",4,4,4,4,4,20,0.425


In [17]:
max_rating_df=data[data["totalratings"]==25]

In [18]:
max_rating_df=max_rating_df.reset_index(drop=True)
max_rating_df

,Car Model,Review,Style,Comfort,Fuel Economy,Performance,Value for Money,totalratings,positivity_in_review
0,2021 United Alpha 1.0 Manual,Exterior: Beautiful Look from front and back b...,5,5,5,5,5,25,0.296
1,2009 Honda City 1.3 i-VTEC Prosmatec,Good car but style is old and fuel economy is ...,5,5,5,5,5,25,0.128
2,2009 Toyota Corolla Altis SR Cruisetronic 1.8,Good car but style is old and fuel economy is ...,5,5,5,5,5,25,0.128
3,2018 Suzuki Cultus VXL,Exterior is very good dynamic shape Interior i...,5,5,5,5,5,25,0.342
4,2019 Suzuki Cultus VXL,I had this car for 5 years. Its great value fo...,5,5,5,5,5,25,0.213
...,...,...,...,...,...,...,...,...,...
623,Daihatsu Cuore,"Its a very beautiful car, drive is much much b...",5,5,5,5,5,25,0.117
624,Suzuki Cultus,"Its economical, spacious, 4 cylinder powerfull...",5,5,5,5,5,25,0.229
625,Chery QQ,I HAV'NT DRIVEN THIS CAR BUT I THINK THIS CAR ...,5,5,5,5,5,25,0.153
626,Daihatsu Cuore,I feel it's an amazing ride inspired by its ol...,5,5,5,5,5,25,0.171


In [19]:
max_rating_df.iloc[max_rating_df['positivity_in_review'].idxmax()]

Car Model                                      2018 Suzuki Vitara GL+ 1.6
Review                  this car is expensive but excellent condition ...
Style                                                                   5
Comfort                                                                 5
Fuel Economy                                                            5
Performance                                                             5
Value for Money                                                         5
totalratings                                                           25
positivity_in_review                                                0.563
Name: 256, dtype: object

In [20]:
max_rating_df.head(10)

,Car Model,Review,Style,Comfort,Fuel Economy,Performance,Value for Money,totalratings,positivity_in_review
0,2021 United Alpha 1.0 Manual,Exterior: Beautiful Look from front and back b...,5,5,5,5,5,25,0.296
1,2009 Honda City 1.3 i-VTEC Prosmatec,Good car but style is old and fuel economy is ...,5,5,5,5,5,25,0.128
2,2009 Toyota Corolla Altis SR Cruisetronic 1.8,Good car but style is old and fuel economy is ...,5,5,5,5,5,25,0.128
3,2018 Suzuki Cultus VXL,Exterior is very good dynamic shape Interior i...,5,5,5,5,5,25,0.342
4,2019 Suzuki Cultus VXL,I had this car for 5 years. Its great value fo...,5,5,5,5,5,25,0.213
5,2021 Honda BR-V i-VTEC S,Don't buy Suzuki they have pathetic after sale...,5,5,5,5,5,25,0.019
6,2020 Suzuki Wagon R Hybrid FZ,Best exterior and interior excellent fuel econ...,5,5,5,5,5,25,0.431
7,2000 Toyota Vitz,Although city is good car and value for money ...,5,5,5,5,5,25,0.249
8,2015 Toyota Corolla XLi VVTi,cars camfort is good but car looks is not so g...,5,5,5,5,5,25,0.159
9,2020 Suzuki Alto VXR,Excellent exterior & interior . With a respect...,5,5,5,5,5,25,0.261


In [21]:
test = max_rating_df.sort_values(['positivity_in_review'], ascending=[False])
test.head(50)

,Car Model,Review,Style,Comfort,Fuel Economy,Performance,Value for Money,totalratings,positivity_in_review
256,2018 Suzuki Vitara GL+ 1.6,this car is expensive but excellent condition ...,5,5,5,5,5,25,0.563
344,2006 Toyota Duet,"The dynamic look, excellent handling, superb d...",5,5,5,5,5,25,0.553
18,2020 DFSK Glory 580 Pro,DFSK glory 580 pro Features are much better th...,5,5,5,5,5,25,0.510
162,2015 Honda N Wgn G Turbo,Modern Exterior. Handling is good ???? but cha...,5,5,5,5,5,25,0.497
108,2020 Toyota Corolla XLi Automatic,Awesome Exterior and Interior . Outclass perfo...,5,5,5,5,5,25,0.478
385,2007 Honda City i-DSI,exterior good and sporty good responsive engin...,5,5,5,5,5,25,0.473
30,2014 Honda Vezel Hybrid Z Honda Sensing,"Exterior is awesome , I previously used GLI an...",5,5,5,5,5,25,0.470
263,2007 Honda Airwave M S Package,Overall its good and a nice family car I was a...,5,5,5,5,5,25,0.468
315,2018 Suzuki Cultus Auto Gear Shift,. Attractive exterior . Comfortable interior ....,5,5,5,5,5,25,0.463
532,Mercedes Benz C Class C180 CGI,Altis is the best 2012 1.6. Thanks toyota i am...,5,5,5,5,5,25,0.461


In [22]:
hondacity_analysis = max_rating_df.loc[max_rating_df['Car Model '].str.contains("Honda City", case=False)]["positivity_in_review"].mean()
print("Honda City positivity")
print(hondacity_analysis)
suzukicultus_analysis = max_rating_df.loc[max_rating_df['Car Model '].str.contains("Suzuki Cultus", case=False)]["positivity_in_review"].mean()
print("Suzuki Cultus positivity")
print(suzukicultus_analysis)
toyotacorrola_analysis = max_rating_df.loc[max_rating_df['Car Model '].str.contains("Toyota Corolla", case=False)]["positivity_in_review"].mean()
print("Toyota Corolla positivity")
print(toyotacorrola_analysis)
hondavezel_analysis = max_rating_df.loc[max_rating_df['Car Model '].str.contains("Toyota Corolla", case=False)]["positivity_in_review"].mean()
print("honda vezel positivity")
print(hondavezel_analysis)
hondacivic_analysis = max_rating_df.loc[max_rating_df['Car Model '].str.contains("Toyota Corolla", case=False)]["positivity_in_review"].mean()
print("Honda Civic positivity")
print(hondacivic_analysis)
suzukioverall_analysis = max_rating_df.loc[max_rating_df['Car Model '].str.contains("suzuki", case=False)]["positivity_in_review"].mean()
print("Suzuki overall positivity")
print(suzukioverall_analysis)
hondaoverall_analysis = max_rating_df.loc[max_rating_df['Car Model '].str.contains("Honda", case=False)]["positivity_in_review"].mean()
print("honda overall positivity")
print(hondaoverall_analysis)
toyotaoverall_analysis = max_rating_df.loc[max_rating_df['Car Model '].str.contains("Toyota", case=False)]["positivity_in_review"].mean()
print("toyota overall positivity")
print(toyotaoverall_analysis)

Honda City positivity
0.2130555555555555
Suzuki Cultus positivity
0.17327906976744184
Toyota Corolla positivity
0.18539215686274507
honda vezel positivity
0.18539215686274507
Honda Civic positivity
0.18539215686274507
Suzuki overall positivity
0.188156976744186
honda overall positivity
0.21008125000000014
toyota overall positivity
0.1892337662337662
